In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import  confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = "/kaggle/input/telco-customer-churnn/Telco_customer_churn_adapted_v2.xlsx"
df = pd.read_excel(dataset)
df['Churn Label'].value_counts()

In [ ]:
df['Churn Label'].info()

In [ ]:
df.info()

In [ ]:
duplicate_values = df['Customer ID'].duplicated()
print(duplicate_values)

In [ ]:
df.tail(20)

In [ ]:
df_nodup = df.drop_duplicates(subset=['Customer ID'], keep='first')
print(df_nodup)
df_nodup.info()

In [ ]:
#Encoding 'object' values
# Location
df_location = df['Location'].tolist()
encodeLocation = []
for location in df_location:
    Loc = location.encode("utf-8")
    intLoc = int.from_bytes(Loc, byteorder = 'big')
    encodeLocation.append(intLoc)

df_Loc = pd.Series(encodeLocation, name='Loc')
df.reset_index(drop=True, inplace=True)
df_encoded = pd.concat([df_Loc, df], axis=1)

#Device Class
df_device_class = df['Device Class'].tolist()
encodeDC = []
for device_class in df_device_class:
        DeviceClass = device_class.encode("utf-8")
        intDeviceClass = int.from_bytes(DeviceClass, byteorder = 'big')
        encodeDC.append(intDeviceClass)
    
df_dc = pd.Series(encodeDC, name = "device_class_encoded")
df_encoded = pd.concat([df_encoded, df_dc], axis = 1)

#Games Product
df_games_product = df['Games Product'].tolist()
encodeGP = []
for games_product in df_games_product:
        GamesProduct = games_product.encode("utf-8")
        intGamesProduct = int.from_bytes(GamesProduct, byteorder = 'big')
        encodeGP.append(intGamesProduct)
    
df_gp = pd.Series(encodeGP, name = "games_product_encoded")
df_encoded = pd.concat([df_encoded, df_gp], axis = 1)

#Music Product
df_music_product = df['Music Product'].tolist()
encodeMP = []
for music_product in df_music_product:
        MusicProduct = music_product.encode("utf-8")
        intMusicProduct = int.from_bytes(MusicProduct, byteorder = 'big')
        encodeMP.append(intMusicProduct)
    
df_mp = pd.Series(encodeMP, name = "music_product_encoded")
df_encoded = pd.concat([df_encoded, df_mp], axis = 1)

#Education Product
df_edu_product = df['Education Product'].tolist()
encodeEP = []
for edu_product in df_edu_product:
        EduProduct = edu_product.encode("utf-8")
        intEduProduct = int.from_bytes(EduProduct, byteorder = 'big')
        encodeEP.append(intEduProduct)
    
df_ep = pd.Series(encodeEP, name = "edu_product_encoded")
df_encoded = pd.concat([df_encoded, df_ep], axis = 1)

#Video Product
df_video_product = df['Video Product'].tolist()
encodeVP = []
for video_product in df_video_product:
        VideoProduct = video_product.encode("utf-8")
        intVideoProduct = int.from_bytes(VideoProduct, byteorder = 'big')
        encodeVP.append(intVideoProduct)
    
df_vp = pd.Series(encodeVP, name = "video_product_encoded")
df_encoded = pd.concat([df_encoded, df_vp], axis = 1)

#Use MyApp
df_use_myapp = df['Use MyApp'].tolist()
encodeUMP = []
for use_myapp in df_use_myapp:
        UseMyApp = use_myapp.encode("utf-8")
        intUseMyApp = int.from_bytes(UseMyApp, byteorder = 'big')
        encodeUMP.append(intUseMyApp)
    
df_ump = pd.Series(encodeUMP, name = "use_myapp_encoded")
df_encoded = pd.concat([df_encoded, df_ump], axis = 1)

#Payment Method
df_payment_met = df['Payment Method'].tolist()
encodePM = []
for payment_met in df_payment_met:
        PaymentMet = payment_met.encode("utf-8")
        intPaymentMet = int.from_bytes(PaymentMet, byteorder = 'big')
        encodePM.append(intPaymentMet)
    
df_pm = pd.Series(encodePM, name = "payment_met_encoded")
df_encoded = pd.concat([df_encoded, df_pm], axis = 1)

print(df_encoded)


In [ ]:
#Call Center dan Churn ubah Yes ke 1 dan No ke 0
#Call Center
df_call_center = df['Call Center'].tolist()
encodeCC = []
for i in range(0,(len(df_call_center))):
    if df_call_center[i] == "Yes":
        encodeCC.insert(i, 1)
    elif df_call_center[i] == "No":
        encodeCC.insert(i, 0)
df_cc = pd.Series(encodeCC, name = "call_center_encoded")
df_encoded = pd.concat([df_encoded, df_cc], axis = 1)

#Churn Label
df_churn_label = df['Churn Label'].tolist()
encodeCL = []
for i in range(0,(len(df_churn_label))):
    if df_churn_label[i] == "Yes":
        encodeCL.insert(i, 1)
    elif df_churn_label[i] == "No":
        encodeCL.insert(i, 0)
df_cL = pd.Series(encodeCL, name = "churn_label_encoded")
df_encoded = pd.concat([df_encoded, df_cL], axis = 1)

print(df_encoded)

In [ ]:
df_encoded = pd.DataFrame(df_encoded).drop(['Location', 'Device Class', 'Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp', 'Payment Method', 'Churn Label'], axis = 1)
print(df_encoded)

In [ ]:
# Correlation matrix
corr_mat = df_encoded.corr()
sns.set(font_scale = 1.2)
plt.figure(figsize=(20,10))
sns.heatmap(corr_mat, cmap='coolwarm', annot=True)
plt.show()

In [ ]:
df_encoded = pd.DataFrame(df_encoded).drop(['Loc','Longitude','Latitude'], axis=1)
print(df_encoded)

In [ ]:
label = df_encoded['churn_label_encoded']
label.reset_index(drop=True, inplace=True)
clean_df = df_encoded.drop(['churn_label_encoded'], axis=1)
clean_df.reset_index(drop=True, inplace=True)

y = label
X = clean_df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Oversampling the minority
oversampler = RandomOverSampler(sampling_strategy='minority')
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train_scaled, y_train)

In [ ]:
# Modelling with Random Forest
rfm = RandomForestClassifier(n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='auto',
    bootstrap=True) #default parameters
rfm.fit(X_train_oversampled, y_train_oversampled)
rfm_pred = rfm.predict(X_test_scaled)

print('Random Forest initial Performance:')
print('----------------------------------')
print('Accuracy        : ', metrics.accuracy_score(y_test, rfm_pred))
print('F1 Score        : ', metrics.f1_score(y_test, rfm_pred))
print('Precision       : ', metrics.precision_score(y_test, rfm_pred))
print('Recall          : ', metrics.recall_score(y_test, rfm_pred))
print('Confusion Matrix:\n ', confusion_matrix(y_test, rfm_pred))

In [ ]:
# Search the best hyperparameter
rfm_param_grid = {'n_estimators': [10,80,200,800,2000,5000], 
                  'max_features': ['sqrt', 'log2'],
                  'max_depth': [10,20,30,40,50,100],
                  'min_samples_split': [2,5,10,20],
                  'min_samples_leaf': [1,2,7,15]} 

rfm_grid = RandomizedSearchCV(RandomForestClassifier(),
                        rfm_param_grid,
                        n_iter=100,
                        n_jobs= -1)

In [ ]:
rfm_grid.fit(X_train_oversampled, y_train_oversampled)

In [ ]:
rfm_grid.best_params_

In [ ]:
rfm_grid.best_estimator_

In [ ]:
# tuning the best hyperparameter
rfm = RandomForestClassifier(n_estimators=5000,
    criterion='gini',
    max_depth=100,
    min_samples_split=5,
    min_samples_leaf=15,
    max_features='log2',
    bootstrap=True) #default parameters
rfm.fit(X_train_oversampled, y_train_oversampled)
rfm_pred = rfm.predict(X_test_scaled)

print('Random Forest initial Performance:')
print('----------------------------------')
print('Accuracy        : ', metrics.accuracy_score(y_test, rfm_pred))
print('F1 Score        : ', metrics.f1_score(y_test, rfm_pred))
print('Precision       : ', metrics.precision_score(y_test, rfm_pred))
print('Recall          : ', metrics.recall_score(y_test, rfm_pred))
print('Confusion Matrix:\n ', confusion_matrix(y_test, rfm_pred))

In [ ]:
# AUC Score and ROC Curve
auc_score = roc_auc_score(y_test, rfm_pred)
print("AUC Score:", auc_score)
fpr, tpr, thresholds = roc_curve(y_test, rfm_pred)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="ROC Curve (AUC = {:.2f})".format(auc_score))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()